### Limitations:

* Works for artihmetic expression
    * operators supported: + - * / % ~(unary minus)
* Works for boolean expression
    * operators supported: && || ! < <= == != >= >
* Works for if and if...else statments
* Works for while loops
* Don't work for very complex instructions    

In [119]:
class ThreeAddressCodeGen:
    quadraple = []
    curr_temp = 0
    lab = 0
    binaryop = ['+','-','*','/','%']
    unaryop = ['~']
    booleanop = ['||','&&','!']
    relop = ['<','<=','==','!=','>=','>']
    
    # Generate a temporary variable
    def GenTemp(self):                  
        self.curr_temp = self.curr_temp + 1
        return 't'+str(self.curr_temp)
    
    # Generate an Label
    def GenLabel(self):            
        self.lab = self.lab + 1
        return 'L'+str(self.lab)
    
    # Generate code for aithmetic instructions
    def arithmetic(self,code):  
        precedence = {'~':3,'*':2,'/':2,'%':2,'+':1,'-':1}
        operators = self.binaryop + self.unaryop
        tokens = code.split(' ') + [')']
        stack = ['(']
        postfix = []
        for i in tokens[2:]:
            if i in operators:
                if len(stack) == 0 or stack[-1] == '(' or (precedence[i] > precedence[stack[-1]]):
                    stack.append(i)
                else:
                    while stack[-1] in operators and precedence[stack[-1]] >= precedence[i]:
                        postfix.append(stack.pop())
                    stack.append(i)
            elif i == ')':
                while stack[-1] != '(':
                    postfix.append(stack.pop())
                stack.pop()
            elif i == '(':
                stack.append(i)
            else:
                postfix.append(i) 

        operands = []
        res = tokens[0]
        for i in postfix:
            if i in operators:
                op = i
                if op in self.binaryop:
                    arg2 = operands.pop()
                    arg1 = operands.pop()
                    self.quadraple.append(' '.join([self.GenTemp(),':=',arg1,op,arg2]))
                else:
                    arg1 = operands.pop()
                    self.quadraple.append(' '.join([self.GenTemp(),':=',op,arg1]))
                operands.append('t'+str(self.curr_temp))
            else:
                operands.append(i)
        self.quadraple.append(' '.join([res,':=','t'+str(self.curr_temp)]))
    
    # Generate code for boolean instructions
    def boolean(self,code,true_label,false_label):
        stack = []
        tokens = code.split(' ')
        quad = []
        for i,j in enumerate(tokens):
            if j in self.relop:
                quad.append(['if'] + tokens[i-1:i+2] + ['goto'])
            if j in self.booleanop:
                stack.append(j)

        for i,j in enumerate(stack):

            if j == '||':
                quad[i].append(true_label)        
            elif j == '&&':
                quad[i][0] = quad[i][0] + 'False'
                quad[i].append(false_label)
            else:
                temp = true_label
                true_label = false_label
                false_label = temp

        if j == '||':
            quad[-1].append(true_label)        
        elif j == '&&':
            quad[-1][0] = quad[-1][0] + 'False'
            quad[-1].append(false_label)
        else:
            temp = true_label
            true_label = false_label
            false_label = temp

        for i in quad:
            self.quadraple.append(' '.join(i))
    
    # Generate code for if, if_else, while instructions
    def GenCode(self,codeblock):
        i = 0
        flage = False
        while i < len(codeblock):
            code = codeblock[i]
            if 'if' in code:
                boolexpr = code.split('( ')[1].split(' )')[0]
                true_label = self.GenLabel()
                false_label = self.GenLabel()
                self.boolean(code,true_label,false_label)
                self.quadraple.append(true_label+' : ')
            elif 'else' in code:
                flage = True
                self.quadraple.append(false_label+' : ')
            elif '}' in code:
                if flage:
                    flage = False
                    i = i+1
                    continue
                self.quadraple.append(false_label+' : ')
            elif 'while' in code:
                boolexpr = code.split('( ')[1].split(' )')[0]
                true_label = self.GenLabel()
                false_label = self.GenLabel()
                self.quadraple.append(true_label+' : ')
                self.boolean(code,true_label,false_label)
            else:
                self.arithmetic(code)
            i = i+1
            
    # Print 3-address code        
    def Print(self):
        print('The 3-address code is : \n')
        for i in self.quadraple:
            print('\t' + i)

In [120]:
codeblock = ["var1 = a + b / ( c * d )",
             "var2 = b * ~ c + b * ~ c",
             "while ( x < 100 || x > 200 && x != y ) {",
             "x = x + var1",
             "}",
             "if ( x < 100 || x > 200 && x != y ) {",
             "x = x + var2",
             "}",
             "else {",
             "x = x - var2",
             "}"]

In [121]:
print('\nThe High Level Code is : \n')
for i in codeblock:
    print('\t' + i)


The High Level Code is : 

	var1 = a + b / ( c * d )
	var2 = b * ~ c + b * ~ c
	while ( x < 100 || x > 200 && x != y ) {
	x = x + var1
	}
	if ( x < 100 || x > 200 && x != y ) {
	x = x + var2
	}
	else {
	x = x - var2
	}


In [122]:
x = ThreeAddressCodeGen()
x.GenCode(codeblock)
x.Print()

The 3-address code is : 

	t1 := c * d
	t2 := b / t1
	t3 := a + t2
	var1 := t3
	t4 := ~ c
	t5 := b * t4
	t6 := ~ c
	t7 := b * t6
	t8 := t5 + t7
	var2 := t8
	L1 : 
	if x < 100 goto L1
	ifFalse x > 200 goto L2
	ifFalse x != y goto L2
	t9 := x + var1
	x := t9
	L2 : 
	if x < 100 goto L3
	ifFalse x > 200 goto L4
	ifFalse x != y goto L4
	L3 : 
	t10 := x + var2
	x := t10
	L4 : 
	L4 : 
	t11 := x - var2
	x := t11
